# Aula 9 - Parte 2 - Analise de Sentimento

Esse notebook descreve o passo a passo para aplicar o classificador criado anteriormente para gerar a classificação dos comentários do Youtube em positivo, negativo ou neutro.

## 1. Carregar base do Youtube

In [ ]:
import pandas as pd

In [ ]:
dados_comentarios = pd.read_csv('todos_comentarios.csv', sep=';', encoding='utf-8')

In [ ]:
def carregar_dados(arquivo):
    dados = pd.read_csv(arquivo, sep=';', encoding='utf-8')
    
    base = []
    for i in range(len(dados)):
        base.append((dados.textDisplay.loc[i], None))
    return base[1:]

In [ ]:
dados_comentarios = carregar_dados('todos_comentarios.csv')

In [ ]:
dados_comentarios

In [ ]:
print(len(dados_comentarios))

## 2. Carregar classificador Naive Bayes

In [ ]:
import pickle

In [ ]:
def carregar_modelo(nome_arquivo):
    nome = str(nome_arquivo) + '.pickle'
    try:
        arquivo = open(nome, "rb")
        modelo = pickle.load(arquivo)
        arquivo.close()
        return modelo
    except Exception as e:
        return e

In [ ]:
classificador = carregar_modelo('naivebayes')

In [ ]:
classificador

## 3. Realizando a classificação para um comentário

Para que a nova frase seja classificada os seguintes passos devem ser realizados:
- Remover as stopwords
- Aplicar o Stemming
- Recuperar as características
- Realizar a classificação

Para facilitar o uso dessas funções, elas foram adicionadas em um arquivo chamda ```utils.py```. Para utiliza-las, basta importar o arquivo.

In [ ]:
import nltk

In [ ]:
from importlib import reload

In [ ]:
import utils

In [ ]:
reload(utils)

In [ ]:
texto = 'eu sinto amor por voce'

In [ ]:
dados_comentarios[0]

In [ ]:
frase = utils.estruturar_dados([dados_comentarios[0]])

In [ ]:
frase

In [ ]:
frase[0][0]

In [ ]:
resultado = utils.classificar_texto(classificador, frase[0][0])

In [ ]:
resultado

## 4. Realizando a classificação para TODOS comentários

In [ ]:
df_classificados = pd.DataFrame(columns=['texto', 'classe', 'prob_pos', 'prob_neg', 'prob_neu', 'emoticon'])
df_classificados

In [ ]:
from time import time

In [ ]:
print(len(dados_comentarios))

Para classificar mais de 167mil comentários irá demorar cerca de 2 horas e 30 minutos, dependendo da sua máquina.

In [ ]:
ti = time()
for i in range(len(dados_comentarios)):
    if i % 5000 == 0:
        print(i, end=" ")
    
    frase = utils.estruturar_dados([dados_comentarios[i]])
    resultado = utils.classificar_texto(classificador, frase[0][0])
    
    if resultado['classe'] == 'positivo':
        emoticon = u"\U0001F642"
    elif resultado['classe'] == 'neutro':
        emoticon = u"\U0001F610"
    elif resultado['classe'] == 'negativo':
        emoticon = u"\U0001F641"
    
    df_classificados.loc[i] = [
        dados_comentarios[i][0], 
        resultado['classe'],
        resultado['positivo'],
        resultado['neutro'],
        resultado['negativo'],
        emoticon]
tf = time() - ti
print("\nTempo total {} em segundos".format(tf))

In [ ]:
df_classificados

In [ ]:
%%time
df_classificados.to_csv('todos_comentarios_classificados.csv', sep=';', header=True, index=False, encoding='utf-8')